In [ ]:
# Download YOLOv7 repository and install requirements

# %cd /content/gdrive/MyDrive/GoogleColabProjects/oai/yolo_localization
!git clone https://github.com/WongKinYiu/yolov7
%cd yolov7
!pip install -r requirements.txt

In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="LedIXO6IeBdnpZKATilQ")
project = rf.workspace("minisummercamp2023").project("tka-localization")
dataset = project.version(1).download("yolov7")

In [ ]:
%%bash
wget -P /content/yolov7 https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7.pt

In [ ]:
!python train.py --batch 16 --cfg cfg/training/yolov7.yaml --epochs 100 --data /content/yolov7/TKA-Localization-1/data.yaml --weights 'yolov7.pt' --device 0

In [ ]:
# Run evaluation
!python detect.py --weights runs/train/exp3/weights/best.pt --conf 0.75 --source TKA-Localization-1/test/images --save-txt --save-conf

In [ ]:
! pip install pybboxes
import pybboxes as pbx

In [81]:
exp= "exp5"
annot_name="04157002_jpg.rf.d3fac05311bf219a9854b0bf796a533e.txt"
ground_truth_dir = f"/content/yolov7/TKA-Localization-1/test/labels/{annot_name}"
exp_dir = f"/content/yolov7/runs/detect/{exp}/labels/{annot_name}"

conf_thresh = 0.7
W, H = 640, 640

with open(ground_truth_dir, "r") as f:
    cls = {}
    lines = f.readlines()
    for f in lines:
        c, x, y, w, h = [float(x) for x in f.split()]
        cls[c] = pbx.convert_bbox((x, y, w, h), from_type="yolo", to_type="voc", image_size=(W, H))

with open(exp_dir, "r") as f:
    preds = {}
    lines = f.readlines()
    for f in lines:
        c, x, y, w, h, conf_lvl = [float(x) for x in f.split()]

        # Don't care about pred if this is true
        if conf_lvl <= conf_thresh:
            continue

        if c in preds:
            p = preds[c]
            p.append(pbx.convert_bbox((x, y, w, h), from_type="yolo", to_type="voc", image_size=(W, H)))
            preds[c] = p
        else:
            preds[c] = [pbx.convert_bbox((x, y, w, h), from_type="yolo", to_type="voc", image_size=(W, H))]



In [82]:
preds

{0.0: [(44, 263, 218, 460), (443, 251, 608, 459)]}

In [89]:
import numpy as np
def calc_intersection_area(bbox1_xmin, bbox2_xmin, bbox1_ymin, bbox2_ymin, bbox1_xmax,
                           bbox2_xmax, bbox1_ymax, bbox2_ymax):
    # determine the (x, y)-coordinates of the intersection rectangle
    inter_xMin = max(bbox1_xmin, bbox2_xmin)
    inter_yMin = max(bbox1_ymin, bbox2_ymin)

    inter_xMax = min(bbox1_xmax, bbox2_xmax)
    inter_yMax = min(bbox1_ymax, bbox2_ymax)

    if inter_xMax < inter_xMin or inter_yMax < inter_yMin:
        return 0.0

    # return the area of intersection rectangle
    return max(0, np.abs(inter_xMax - inter_xMin)) * max(0, np.abs(inter_yMax - inter_yMin))


In [ ]:
for pred, bboxes in preds.items():
    gt = cls[pred]
    if type(gt) == tuple:
        x1_min, y1_min, x2_max, y2_max = cls[pred]

        for bbox in bboxes:
            x3_min, y3_min, x4_max, y4_max = bbox
            intersection = calc_intersection_area(x1_min, x3_min, y1_min, y3_min, x2_max,
                            x4_max, y2_max, y4_max)
            annot1_area = np.abs((x2_max - x1_min) * (y2_max - y1_min))
            annot2_area = np.abs((x4_max - x3_min) * (y4_max - y3_min))
            iou = intersection / (annot1_area + annot2_area - intersection)
            print(pred, iou)
    else:
        max_iou = 0
        for gt in cls[pred]:
            x1_min, y1_min, x2_max, y2_max = gt
            for bbox in bboxes:
                x3_min, y3_min, x4_max, y4_max = bbox
                intersection = calc_intersection_area(x1_min, x3_min, y1_min, y3_min, x2_max,
                                x4_max, y2_max, y4_max)

                annot1_area = np.abs((x2_max - x1_min) * (y2_max - y1_min))
                annot2_area = np.abs((x4_max - x3_min) * (y4_max - y3_min))

                iou = intersection / (annot1_area + annot2_area - intersection)
                if iou > max_iou:
                    max_iou = iou
        print(pred, iou)
